In [19]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
%load_ext line_profiler # Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [42]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [60]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
celeri.plot_block_labels(segment, block, station, closure)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment) # This function need work
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [128]:
def latitude_to_colatitude(lat):
    """
    Convert from latitude to colatitude
    NOTE: Not sure why I need to treat the scalar case differently but I do.
    """
    if lat.size == 1:  # Deal with the scalar case
        if lat >= 0:
            lat = 90.0 - lat
        elif lat < 0:
            lat = -90.0 - lat
    else:  # Deal with the array case
        lat[np.where(lat >= 0)[0]] = 90.0 - lat[np.where(lat >= 0)[0]]
        lat[np.where(lat < 0)[0]] = -90.0 - lat[np.where(lat < 0)[0]]
    return lat


def get_block_centroid(segment, block_idx):
    """
    Calculate centroid of a block based on boundary polygon
    We take all block vertices (including duplicates) and estimate
    the centroid by taking the average of longitude and latitude
    weighted by the length of the segment that each vertex is 
    attached to.
    """
    segments_with_block_idx = np.union1d(np.where(segment.west_labels == 0)[0], np.where(segment.east_labels == 0)[0])
    lon0 = np.concatenate((segment.lon1[segments_with_block_idx], segment.lon2[segments_with_block_idx]))
    lat0 = np.concatenate((segment.lat1[segments_with_block_idx], segment.lat2[segments_with_block_idx]))
    lengths = np.concatenate((segment.length[segments_with_block_idx], segment.length[segments_with_block_idx]))
    block_centroid_lon = np.average(lon0, weights=lengths)
    block_centroid_lat = np.average(lat0, weights=lengths)
    return block_centroid_lon, block_centroid_lat


def get_strain_rate_displacements(station, segment, block_idx, strain_rate_lon_lon, strain_rate_lat_lat, strain_rate_lon_lat):
    """
    Calculate displacements within a signble block due to three
    strain rate components within that block.  Velocities will be
    zero on all blocks other than than the the current block.

    Equations are from Savage (2001) and expressed concisely in McCaffrey (2005)
    In McCaffrey (2005) these are the two unnumbered equations at the bottom
    of page 2
    """

    # Find block centroids and convert to colatitude and radians
    block_centroid_lon, block_centroid_lat = get_block_centroid(segment, block_idx)
    block_centroid_lon = np.deg2rad(block_centroid_lon)
    block_centroid_lat = latitude_to_colatitude(block_centroid_lat)
    block_centroid_lat = np.deg2rad(block_centroid_lat)

    # Find indices of stations on the current block and convert to colatitude and radians
    station_block_idx = np.where(station.block_label == block_idx)[0]
    stations_block_lon = station.lon[station_block_idx].to_numpy()
    stations_block_lon = np.deg2rad(stations_block_lon)
    stations_block_lat = station.lat[station_block_idx].to_numpy()
    stations_block_lat = latitude_to_colatitude(stations_block_lat)
    stations_block_lat = np.deg2rad(stations_block_lat)

    # Calculate displacements from homogeneous strain
    u_east = np.zeros(len(station))
    u_north = np.zeros(len(station))
    u_up = np.zeros(len(station)) # Always zero here because we're assuming plane strain on the sphere
    u_east_current_block = strain_rate_lon_lon * (celeri.RADIUS_EARTH * (stations_block_lon - block_centroid_lon) * np.sin(block_centroid_lat)) + strain_rate_lon_lat * (celeri.RADIUS_EARTH * (stations_block_lat - block_centroid_lat))
    u_north_current_block = strain_rate_lon_lat * (celeri.RADIUS_EARTH * (stations_block_lon - block_centroid_lon) * np.sin(block_centroid_lat)) + strain_rate_lat_lat * (celeri.RADIUS_EARTH * (stations_block_lat - block_centroid_lat))
    u_east[station_block_idx] = u_east_current_block
    u_north[station_block_idx] = u_north_current_block
    return u_east, u_north, u_up

# def get_strain_rate_centroid_operator(block, station, segment):
#     """
#     Calculate strain partial derivatives assuming a strain centroid at the center of each block
#     TODO: Return something related to assembly.index???
#     """
block.strain_rate_flag.values[0] = 1  #TODO: REMOVE THIS.  ONLY FOR TESTING
strain_rate_block_idx = np.where(block.strain_rate_flag.to_numpy() > 0)[0]
if strain_rate_block_idx.size > 0:
    block_strain_rate_operator = np.zeros((3 * len(station), 3 * strain_rate_block_idx.size))
    for i in range(strain_rate_block_idx.size):
        # Calculate partials for each component of strain rate
        u_east_lon_lon, u_north_lon_lon, u_up_lon_lon = get_strain_rate_displacements(station, segment, block_idx=strain_rate_block_idx[i], strain_rate_lon_lon=1, strain_rate_lat_lat=0, strain_rate_lon_lat=0)
        u_east_lat_lat, u_north_lat_lat, u_up_lat_lat = get_strain_rate_displacements(station, segment, block_idx=strain_rate_block_idx[i], strain_rate_lon_lon=0, strain_rate_lat_lat=1, strain_rate_lon_lat=0)
        u_east_lon_lat, u_north_lon_lat, u_up_lon_lat = get_strain_rate_displacements(station, segment, block_idx=strain_rate_block_idx[i], strain_rate_lon_lon=0, strain_rate_lat_lat=0, strain_rate_lon_lat=1)

        # TODO: Interleave velocities and insert columns into operator

        # block_strain_rate_operator = block_strain_rate_operator[:, keep_columns]
else:
    block_strain_rate_operator = np.empty(0)

#     # TODO: Write a test here that checks the number of stations on each block that includes
#     # internal strain.  Send an error if any of those blocks have less the command.min_n_stations_strain_rate_block

#     return block_strain_rate_operator, strain_rate_block_idx


block_idx = 0
def plot_strain_rate_components_for_block(closure, segment, station, block_idx):
    plt.figure(figsize=(10, 3))
    plt.subplot(1, 3, 1)
    u_east, u_north, u_up = get_strain_rate_displacements(station, segment, block_idx=block_idx, strain_rate_lon_lon=1, strain_rate_lat_lat=0, strain_rate_lon_lat=0)
    for i in range(closure.n_polygons()):
        plt.plot(closure.polygons[i].vertices[:, 0], closure.polygons[i].vertices[:, 1], "k-", linewidth=0.5)
    plt.quiver(station.lon, station.lat, u_east, u_north, scale=1e7, scale_units="inches", color="r")

    plt.subplot(1, 3, 2)
    u_east, u_north, u_up = get_strain_rate_displacements(station, segment, block_idx=block_idx, strain_rate_lon_lon=0, strain_rate_lat_lat=1, strain_rate_lon_lat=0)
    for i in range(closure.n_polygons()):
        plt.plot(closure.polygons[i].vertices[:, 0], closure.polygons[i].vertices[:, 1], "k-", linewidth=0.5)
    plt.quiver(station.lon, station.lat, u_east, u_north, scale=1e7, scale_units="inches", color="r")

    plt.subplot(1, 3, 3)
    u_east, u_north, u_up = get_strain_rate_displacements(station, segment, block_idx=block_idx, strain_rate_lon_lon=0, strain_rate_lat_lat=0, strain_rate_lon_lat=1)
    for i in range(closure.n_polygons()):
        plt.plot(closure.polygons[i].vertices[:, 0], closure.polygons[i].vertices[:, 1], "k-", linewidth=0.5)
    plt.quiver(station.lon, station.lat, u_east, u_north, scale=1e7, scale_units="inches", color="r")
    plt.show()

plot_strain_rate_components_for_block(closure, segment, station, block_idx)


[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …